# Lecture 4: Password CPA Attack - Introduction

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./4_password_fixed.c')

In [ ]:
import struct
import time
import warnings
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write('p', attempt + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read('p', 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])

In [ ]:
import math
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10
from bokeh.models import Span, Label, BoxAnnotation
from bokeh.layouts import gridplot, row, column

output_notebook()

## Analysis of differences by looking at hamming weights

### Connect data with assembly

We can repeat the plot from last lectures analysis. The assembly code changed of course and we see that there is no more peak when the attempts' character matches.

In [ ]:
traces = [(attempt, capture(attempt, count=10)[0]) for attempt in ('a', 'b', 'i', '\x00', '\xff')]

p = figure(height=500, sizing_mode='stretch_width', x_range=(25, 55))
p.add_tools(CrosshairTool())
for idx, (attempt, trace) in enumerate(traces):
    p.line(
        range(len(trace)),
        abs(trace),
        line_color=Category10_10[idx],
        legend_label=f'abs({attempt.encode("iso-8859-1")})',
    )

for i, (x, label) in enumerate(zip(
    range(29, 80, 4), 
    ('ld r24, Z+', '', 'ld r18, X+', '', 'eor r18, r24', 'or r25, r18')
)):
    if not label:
        continue
    p.add_layout(Span(location=x, dimension='height', line_color='darkslateblue', line_width=30, line_alpha=0.1))
    p.add_layout(Label(x=x, y=p.plot_height, text=label, y_units='screen', x_offset=-15, y_offset=-35,
                       text_align='right', text_color='darkslateblue', angle=math.pi/2))
    for idx, (_, trace) in enumerate(traces):
        p.circle(x, abs(trace)[x], size=10, color=Category10_10[idx])

show(p)

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
There are points in the plot above where all traces are nearly equal and there are points where they differ significantly. Explain this.

</div>

### Displaying Hamming Weights

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [ ]:
traces = [(attempt, capture(attempt, count=10)[0]) for attempt in b'\x00\x01\x03\x07\x0f\x1f\x3f\x7f\xff']

p = figure(title='hw(attempt) vs trace @ ld r18, X+', sizing_mode='stretch_width', plot_height=400)
p.toolbar_location = None
for idx, (attempt, trace) in enumerate(traces):
    p.circle(
        hw(attempt),
        trace[37],
        color=Category10_10[idx],
        size=10,
        legend_label=f'{attempt:02x}',
    )

show(p)

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>
    
Explain the plot above. Take the following principle into account:

> The power consumption of a device is proportional to the hamming weight of the data it processes.

</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 3</div>
    
Make the plot above more vivid:
1. Capture a few hundred (~500) traces each with a randomly chosen input of length 1.
2. Repeat the plot above with the new input. 
    
Side note: The python library `tqdm` provides a nice way of visualizing the progress of a loop:
```python
import tqdm
import tqdm.notebook

for element in tqdm.notebook.tqdm(mylist):
    # Do something with element
```
</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 4</div>

Play around and look how your plot looks like at different positions of the trace, e.g. at `ld r24, Z+`, `eor r18, r24`. Is it clear why?
    
</div>